In [26]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

import yfinance as yf

import boto3

# personal functions from another script
from functions import calendar_features, create_targets, calculate_all_indicators, calculate_all_indicators_optimised

#### PART 1 - Download data

In [27]:
# download of BTC-EUR data from yfinance over a period of 5 years
btc_ticker = yf.Ticker("BTC-EUR")
btc = btc_ticker.history(actions=False, period="5y")

# allow to localize the time zone
btc = btc.tz_localize(None)

# save the raw data in a csv file on local computer
btc.to_csv("rawdata_yfinance_BTC.csv", index=True)

In [28]:
def read_credentials(file_path):
    """_summary_
    a function to recover access id and acess key for aws, stored in a txt file
    Args:
        file_path (string): path where is located the txt files containing credentials
    Returns:
        a list containing credentials
    """
    credentials = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            credentials[key] = value
    return credentials

# Specify the path to credentials file
credentials_path = "access.txt"

# Read credentials from the file
credentials = read_credentials(credentials_path)

# Access individual credentials
access_key = credentials["access_key"]
secret_key = credentials["secret_key"]

In [29]:
# saving files to S3
session = boto3.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key)

# create a "ressource" session
s3 = session.resource("s3")

# upload the csv file to an existing S3 bucket
s3.Bucket("myfinal-project").upload_file(Filename="rawdata_yfinance_BTC.csv", Key="mydata/rawdata_yfinance_BTC.csv")

In [30]:
# Overview of the type of data and potentially nan value for the different columns
btc.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1827 entries, 2018-12-26 to 2023-12-26
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1827 non-null   float64
 1   High    1827 non-null   float64
 2   Low     1827 non-null   float64
 3   Close   1827 non-null   float64
 4   Volume  1827 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 85.6 KB


In [31]:
# overview of some rows, from the end of the dataset.
# we also can see that yfinance gives data for the present day, even if it is not finished.
# we supposed that datas for this day are based on the last known
# datas are ordered by ascending chronological order (from the oldest date to the most recent one)
btc.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2023-12-22,39881.292969,40253.832031,39572.253906,39858.062500,20399842399
2023-12-23,39857.914062,40339.125000,39388.527344,39903.324219,19071527621
2023-12-24,39916.292969,39919.464844,39316.949219,39667.652344,12250301745
2023-12-25,39657.515625,39868.914062,38848.808594,39045.214844,17092267976
2023-12-26,39568.429688,39568.429688,38260.019531,38260.019531,24599017472


#### PART 2 - FEATURE ENGIENNERING

In [32]:
# we apply to our raw data a function that calculate various technical indicators, and add them to a new dataset
# this function is detailed in an another script called functions (imported at the top of this notebook)
btc_completed = calculate_all_indicators(btc)

In [33]:
# overview of job done from the top
btc_completed.head()

,Open,High,Low,Close,Volume,Williams_%R14,ROC_14,rsi_7,rsi_14,rsi_28,...,ATR,Middle Band,Upper Band,Lower Band,OBV_10,CMF_5,VPT_7,A/D Line_3,MFI_14,EOM_14
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-26,3346.593262,3411.186523,3310.973389,3395.228027,4688475417,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-27,3392.916260,3407.929443,3188.728516,3197.248291,4487918526,NaN,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.975837e+09,0.000000,-3.064911
2018-12-28,3195.960693,3459.712158,3178.294922,3429.697021,4922254447,NaN,NaN,54.004033,54.004033,54.004033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.389809e+10,561.412421,-0.941448
2018-12-29,3437.190430,3464.519531,3339.224365,3339.224365,4362951863,NaN,NaN,44.624348,44.624348,44.624348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.844748e+10,-162.883278,0.165639
2018-12-30,3340.951660,3411.224365,3318.955566,3379.538330,4170344379,NaN,NaN,48.602167,48.602167,48.602167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.741784e+10,-68.541689,-0.079221


In [34]:
# overview of job done from the botoom
btc_completed.tail()

,Open,High,Low,Close,Volume,Williams_%R14,ROC_14,rsi_7,rsi_14,rsi_28,...,ATR,Middle Band,Upper Band,Lower Band,OBV_10,CMF_5,VPT_7,A/D Line_3,MFI_14,EOM_14
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,39881.292969,40253.832031,39572.253906,39858.062500,20399842399,-38.263561,-0.660081,57.652680,48.651177,65.522951,...,1416.676563,39159.586133,41992.939258,36326.233008,1.041528e+12,788109.704602,8.096583e+10,-9.398365e+13,951.962805,-3.478983
2023-12-23,39857.914062,40339.125000,39388.527344,39903.324219,19071527621,-31.576800,-2.663987,68.001263,43.926169,65.108435,...,1416.114453,39341.703125,42173.932031,36509.474219,1.042264e+12,761488.992765,8.098627e+10,-9.402694e+13,996.234062,-5.612421
2023-12-24,39916.292969,39919.464844,39316.949219,39667.652344,12250301745,-32.996577,-2.263830,61.753297,44.789569,64.130386,...,1407.455469,39488.848242,42303.759180,36673.937305,1.039312e+12,845852.364394,8.090838e+10,-9.406142e+13,914.750272,-6.694091
2023-12-25,39657.515625,39868.914062,38848.808594,39045.214844,17092267976,-46.407695,-3.987770,63.867001,41.336123,62.910443,...,1339.360938,39504.283594,42183.005469,36825.561719,1.036978e+12,882229.679932,8.066817e+10,-9.409370e+13,311.540848,-7.472949
2023-12-26,39568.429688,39568.429688,38260.019531,38260.019531,24599017472,-65.715316,-0.133885,39.504465,49.670717,61.194827,...,1264.458398,39375.662891,41904.579687,36846.746094,1.033984e+12,890406.478148,8.020379e+10,-9.412966e+13,540.928614,1.276762


In [35]:
print("Our new dataset has {} rows and {} columns.".format(btc_completed.shape[0], btc_completed.shape[1]))
print("We added {} columns to the original dataset".format(btc_completed.shape[1]-5))

Our new dataset has 1827 rows and 77 columns.
We added 72 columns to the original dataset


We apply to our dataset completed two another functions.
These functions are detailed in an another script called functions (imported at the top of this notebook)

In [36]:
# the first one add a few columns about the calendar (month, year, day of week ..)
btc_completed = calendar_features(btc_completed)

In [37]:
btc_completed.tail()

,Open,High,Low,Close,Volume,Williams_%R14,ROC_14,rsi_7,rsi_14,rsi_28,...,OBV_10,CMF_5,VPT_7,A/D Line_3,MFI_14,EOM_14,month,day,year,day_of_week
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,39881.292969,40253.832031,39572.253906,39858.062500,20399842399,-38.263561,-0.660081,57.652680,48.651177,65.522951,...,1.041528e+12,788109.704602,8.096583e+10,-9.398365e+13,951.962805,-3.478983,12,22,2023,4
2023-12-23,39857.914062,40339.125000,39388.527344,39903.324219,19071527621,-31.576800,-2.663987,68.001263,43.926169,65.108435,...,1.042264e+12,761488.992765,8.098627e+10,-9.402694e+13,996.234062,-5.612421,12,23,2023,5
2023-12-24,39916.292969,39919.464844,39316.949219,39667.652344,12250301745,-32.996577,-2.263830,61.753297,44.789569,64.130386,...,1.039312e+12,845852.364394,8.090838e+10,-9.406142e+13,914.750272,-6.694091,12,24,2023,6
2023-12-25,39657.515625,39868.914062,38848.808594,39045.214844,17092267976,-46.407695,-3.987770,63.867001,41.336123,62.910443,...,1.036978e+12,882229.679932,8.066817e+10,-9.409370e+13,311.540848,-7.472949,12,25,2023,0
2023-12-26,39568.429688,39568.429688,38260.019531,38260.019531,24599017472,-65.715316,-0.133885,39.504465,49.670717,61.194827,...,1.033984e+12,890406.478148,8.020379e+10,-9.412966e+13,540.928614,1.276762,12,26,2023,1


In [38]:
# the second one create a few columns with the value to be predicted (target), according to differents horizons of time.
# for example, price of tomorrow(j1), after tomorrow (j2)
btc_completed = create_targets(btc_completed)

In [39]:
print("Our new dataset has now {} rows and {} columns.".format(btc_completed.shape[0], btc_completed.shape[1]))

Our new dataset has now 1827 rows and 85 columns.


In [40]:
# overview of the dataset from the most old days
# we can see that some columns don't have value (Nan). 
# Indeed, it wasn't possible to calculate those indicators as we don't have hostorical values
btc_completed.head()

,Open,High,Low,Close,Volume,Williams_%R14,ROC_14,rsi_7,rsi_14,rsi_28,...,MFI_14,EOM_14,month,day,year,day_of_week,targetvalue_j1,targetvalue_j2,targetvalue_j3,target_night
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-26,3346.593262,3411.186523,3310.973389,3395.228027,4688475417,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,12,26,2018,2,3197.248291,3429.697021,3339.224365,3392.916260
2018-12-27,3392.916260,3407.929443,3188.728516,3197.248291,4487918526,NaN,NaN,0.000000,0.000000,0.000000,...,0.000000,-3.064911,12,27,2018,3,3429.697021,3339.224365,3379.538330,3195.960693
2018-12-28,3195.960693,3459.712158,3178.294922,3429.697021,4922254447,NaN,NaN,54.004033,54.004033,54.004033,...,561.412421,-0.941448,12,28,2018,4,3339.224365,3379.538330,3256.927734,3437.190430
2018-12-29,3437.190430,3464.519531,3339.224365,3339.224365,4362951863,NaN,NaN,44.624348,44.624348,44.624348,...,-162.883278,0.165639,12,29,2018,5,3379.538330,3256.927734,3353.505859,3340.951660
2018-12-30,3340.951660,3411.224365,3318.955566,3379.538330,4170344379,NaN,NaN,48.602167,48.602167,48.602167,...,-68.541689,-0.079221,12,30,2018,6,3256.927734,3353.505859,3483.446045,3380.236084


In [41]:
# overview of the dataset from the most recent days
btc_completed.tail()

,Open,High,Low,Close,Volume,Williams_%R14,ROC_14,rsi_7,rsi_14,rsi_28,...,MFI_14,EOM_14,month,day,year,day_of_week,targetvalue_j1,targetvalue_j2,targetvalue_j3,target_night
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-22,39881.292969,40253.832031,39572.253906,39858.062500,20399842399,-38.263561,-0.660081,57.652680,48.651177,65.522951,...,951.962805,-3.478983,12,22,2023,4,39903.324219,39667.652344,39045.214844,39857.914062
2023-12-23,39857.914062,40339.125000,39388.527344,39903.324219,19071527621,-31.576800,-2.663987,68.001263,43.926169,65.108435,...,996.234062,-5.612421,12,23,2023,5,39667.652344,39045.214844,38260.019531,39916.292969
2023-12-24,39916.292969,39919.464844,39316.949219,39667.652344,12250301745,-32.996577,-2.263830,61.753297,44.789569,64.130386,...,914.750272,-6.694091,12,24,2023,6,39045.214844,38260.019531,NaN,39657.515625
2023-12-25,39657.515625,39868.914062,38848.808594,39045.214844,17092267976,-46.407695,-3.987770,63.867001,41.336123,62.910443,...,311.540848,-7.472949,12,25,2023,0,38260.019531,NaN,NaN,39568.429688
2023-12-26,39568.429688,39568.429688,38260.019531,38260.019531,24599017472,-65.715316,-0.133885,39.504465,49.670717,61.194827,...,540.928614,1.276762,12,26,2023,1,NaN,NaN,NaN,NaN


In [42]:
#save the complete dataset with all the columns in local computer
btc_completed.to_csv("btc_completed.csv", index=True)

In [43]:
# upload the csv file to an existing S3 bucket
s3.Bucket("myfinal-project").upload_file(Filename="btc_completed.csv", Key="mydata/btc_completed.csv")

#### PART 3 - Data analyse

In [44]:
# correlation matrix
# too many values to be visualized with a heatmap
# we will used a matrix with value sorted by importance
corr_btc = btc.iloc[:,:-3].corr()

# we choose to see the correlation between all the features and one target, in this case the proce of tomorrow
corr_btc = abs(corr_btc.loc[:,"targetvalue_j1"]).sort_values(ascending=True)

# we have a look at the 20th rows with lower correlation coefficient
corr_btc.head(20)

day_of_week             0.000188
Z_Score_20              0.003133
Percent_Return_1        0.003804
Stochastic_K3           0.006409
Aroon_Down_16           0.007140
MACD_Histogram          0.011178
day                     0.012589
Stochastic_D3           0.014272
Percent_Return_5        0.014795
Chaikin_Oscillator_3    0.015212
LogReturn_5             0.016433
MFI_14                  0.018402
Negative_VI_7           0.018642
Positive_VI_7           0.021191
rsi_14                  0.021620
rsi_28                  0.022343
Percent_Return_10       0.023482
rsi_7                   0.025157
kurtosis_5              0.026684
LogReturn_10            0.027548
Name: targetvalue_j1, dtype: float64

In [45]:
# we have a look at the 15th rows with higher correlation coefficient
corr_btc.tail(15)

midlle_band          0.984014
Donchian_Upper_20    0.984112
Upper_Band_20        0.984550
Donchian_Lower_10    0.990131
ema_15               0.990752
Donchian_Upper_10    0.990837
ALMA_10              0.991355
Tenkan_sen           0.993130
ema_8                0.994689
Open                 0.996004
Low                  0.996795
High                 0.997087
ema_3                0.997297
Close                0.997946
targetvalue_j1       1.000000
Name: targetvalue_j1, dtype: float64

We can see at this time that some features don't really have a correlation with the target (in our case price at j+1)
It will be possible to make a first selection of available features in order to reduce the number of them given to our models.
We made the choice to not eliminate features at this moment.
This selction will be done after training the model.

In [46]:
# create a daframe with correlation coefficient for each target
coef_compare = pd.DataFrame()
for i in range(1,5,1) :
    corr_btc = btc_completed.corr().iloc[:,-i]
    coef_compare[btc_completed.columns[-i]] = abs(corr_btc)

In [47]:
# overview of lower correlation coefficient
coef_compare.sort_values(ascending= True, by="targetvalue_j1").head(15)

,target_night,targetvalue_j3,targetvalue_j2,targetvalue_j1
day_of_week,0.000894,0.001109,0.000051,0.000188
Z_Score_20,0.001357,0.006611,0.005141,0.003133
Percent_Return_1,0.005584,0.006452,0.005542,0.003804
Stochastic_K3,0.005618,0.011004,0.009560,0.006409
Aroon_Down_16,0.006432,0.011187,0.008891,0.007140
MACD_Histogram,0.011877,0.009800,0.010131,0.011178
day,0.013563,0.011219,0.011864,0.012589
Stochastic_D3,0.012246,0.017820,0.017082,0.014272
Percent_Return_5,0.013126,0.018558,0.017404,0.014795
Chaikin_Oscillator_3,0.014869,0.015409,0.015544,0.015212


We can see that these coefficients are very closed according to the target.
In this context, we will make the choice to dismiss some of them due to their lesser importance.
This will allow us to optimize the process.

We removed the following features :
- all the features regarding the calendar
- Chainlin_Oscillator_3
- Percent_return_1 & Percent_return_5
- stochastic_k3 and stochastic_d3
- MFI_14
- CCI_20
- logreturn_5
- Negative_VI_7 & Positive_VI_7

We kept these ratio as they come with another indicators :
- macd_histogram
- Aroon_Down_16

In [48]:
# function to create our complete dataframe after optimisation
def dataset_completion(dataset_path) :
    """
    Args:
        dataset (dataframe - csv file): original dataset saved with columns (Low, High, Open, Close, Volume)
    Returns:
        a dataset with new columns added (technical indicators and targets)
    """
    dataset = pd.read_csv(dataset_path, index_col=0)
    dataset = calculate_all_indicators_optimised(dataset)
    dataset = create_targets(dataset)
    return dataset

In [56]:
btc_completed_optimized = dataset_completion("rawdata_yfinance_BTC.csv")

In [57]:
# check the job
btc_completed_optimized.head(10)

,Open,High,Low,Close,Volume,Williams_%R14,ROC_14,rsi_7,rsi_14,rsi_28,...,VIX_21,OBV_10,CMF_5,VPT_7,A/D Line_3,EOM_14,targetvalue_j1,targetvalue_j2,targetvalue_j3,target_night
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-26,3346.593262,3411.186523,3310.973389,3395.228027,4688475417,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3197.248291,3429.697021,3339.224365,3392.916260
2018-12-27,3392.916260,3407.929443,3188.728516,3197.248291,4487918526,NaN,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,-8.975837e+09,-3.064911,3429.697021,3339.224365,3379.538330,3195.960693
2018-12-28,3195.960693,3459.712158,3178.294922,3429.697021,4922254447,NaN,NaN,54.004033,54.004033,54.004033,...,NaN,NaN,NaN,NaN,-1.389809e+10,-0.941448,3339.224365,3379.538330,3256.927734,3437.190430
2018-12-29,3437.190430,3464.519531,3339.224365,3339.224365,4362951863,NaN,NaN,44.624348,44.624348,44.624348,...,NaN,NaN,NaN,NaN,-1.844748e+10,0.165639,3379.538330,3256.927734,3353.505859,3340.951660
2018-12-30,3340.951660,3411.224365,3318.955566,3379.538330,4170344379,NaN,NaN,48.602167,48.602167,48.602167,...,NaN,NaN,NaN,NaN,-2.741784e+10,-0.079221,3256.927734,3353.505859,3483.446045,3380.236084
2018-12-31,3380.236084,3366.611084,3242.279785,3256.927734,4056771164,NaN,NaN,39.887752,39.887752,39.887752,...,NaN,NaN,223.312716,NaN,-3.581122e+10,-0.435102,3353.505859,3483.446045,3368.355713,3260.419922
2019-01-01,3260.419922,3358.304932,3235.652832,3353.505859,3772904282,NaN,NaN,47.326886,47.326886,47.326886,...,NaN,NaN,309.586016,NaN,-4.381124e+10,-0.403039,3483.446045,3368.355713,3385.297607,3358.422119
2019-01-02,3358.422119,3481.464111,3323.570312,3483.446045,4633091490,NaN,NaN,54.845312,54.845312,54.845312,...,NaN,NaN,367.884979,1.816670e+08,-5.211975e+10,0.168352,3368.355713,3385.297607,3374.292725,3473.871582
2019-01-03,3473.871582,3471.459229,3355.364014,3368.355713,3977171075,NaN,NaN,60.339384,48.689711,48.689711,...,NaN,NaN,429.817700,3.338969e+07,-6.037519e+10,0.187060,3385.297607,3374.292725,3572.601807,3363.948730


In [58]:
# saved the optimized dataset
btc_completed_optimized.to_csv("btc_completed_optimized.csv", index=True)

#### PART 4 - Some data visualization

In [52]:
# Example of visualization of bitcoin prices evolution over the whole period (5 years)
# with a range slider and some ready to use buttons
fig1 = px.line(btc_completed_optimized, x=btc_completed_optimized.index, y="Close", height=800, labels={"Close":"Close price in Eur"})
fig1.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])))
fig1.show()

In [53]:
# another example of visulazization of ema 8 days againt bitcoin prices ove rthe whole period
# with still a range slider
fig2 = px.line(btc_completed_optimized, x=btc_completed_optimized.index, y="Close", height=800, labels={"Close":"Close price in Eur"})
fig2a = px.line(btc_completed_optimized, x=btc_completed_optimized.index, y="ema_8", color_discrete_sequence=["red"])
fig2.add_trace(fig2a.data[0])
fig2.update_xaxes(rangeslider_visible=True)
fig2.show()

In [54]:
# to finish an example of visualization of CC20 over the period
fig3 = px.line(btc_completed_optimized, x=btc_completed_optimized.index, y=["rsi_14", "rsi_7"])
fig3.show()